In [1]:
!ls /dev/tty*

/dev/tty    /dev/tty21	/dev/tty35  /dev/tty49	/dev/tty62    /dev/ttyp2
/dev/tty0   /dev/tty22	/dev/tty36  /dev/tty5	/dev/tty63    /dev/ttyp3
/dev/tty1   /dev/tty23	/dev/tty37  /dev/tty50	/dev/tty7     /dev/ttyp4
/dev/tty10  /dev/tty24	/dev/tty38  /dev/tty51	/dev/tty8     /dev/ttyp5
/dev/tty11  /dev/tty25	/dev/tty39  /dev/tty52	/dev/tty9     /dev/ttyp6
/dev/tty12  /dev/tty26	/dev/tty4   /dev/tty53	/dev/ttyACM0  /dev/ttyp7
/dev/tty13  /dev/tty27	/dev/tty40  /dev/tty54	/dev/ttyGS0   /dev/ttyp8
/dev/tty14  /dev/tty28	/dev/tty41  /dev/tty55	/dev/ttyS0    /dev/ttyp9
/dev/tty15  /dev/tty29	/dev/tty42  /dev/tty56	/dev/ttyS1    /dev/ttypa
/dev/tty16  /dev/tty3	/dev/tty43  /dev/tty57	/dev/ttyS2    /dev/ttypb
/dev/tty17  /dev/tty30	/dev/tty44  /dev/tty58	/dev/ttyS3    /dev/ttypc
/dev/tty18  /dev/tty31	/dev/tty45  /dev/tty59	/dev/ttyTHS1  /dev/ttypd
/dev/tty19  /dev/tty32	/dev/tty46  /dev/tty6	/dev/ttyTHS2  /dev/ttype
/dev/tty2   /dev/tty33	/dev/tty47  /dev/tty60	/dev/ttyp0    /dev/ttypf
/dev/tty2

In [11]:
import os
import sys
import time
import smbus
import math
import numpy as np

from imusensor.MPU9250 import MPU9250

from jetbot import Robot
import PIL

import serial


robot = Robot()


# GPS (UART)

ser = serial.Serial("/dev/ttyACM0") # Address of GPS Device connected via USB
gpgga_info = "$GPGGA," # Extract GPGGA line from GPS
GPGGA_buffer = 0
NMEA_buff = 0

# IMU Address
address = 0x68
bus = smbus.SMBus(0)
imu = MPU9250.MPU9250(bus, address)
imu.begin()
imu.loadCalibDataFromFile("docs/calib_real4.json")


def convert_to_degrees(raw_value):
    decimal_value = raw_value/100.00
    degrees = int(decimal_value)
    mm_mmmm = (decimal_value - int(decimal_value))/0.6
    position = degrees + mm_mmmm
#     position = "%.4f" %(position)
    return position


# Magneto Meter

def currentHeading(): #Get current Headding/Facing direction from Magnetometer (0 to 360 deg)
    
    imu.readSensor()
    imu.computeOrientation()
    
        
    magX = imu.MagVals[0] 
    magY = imu.MagVals[1]
    M_PI = math.pi
    offSet = 30 # Correction Angle
    
    if (magY > 0):
        heading = 90 - math.atan(magX/magY)*(180/M_PI)
        
    elif (magY < 0):
        heading = 270 - math.atan(magX/magY)*(180/M_PI)
        
    elif (magY == 0 and magX < 0):
        heading = 180
        
    elif (magY == 0 and magX > 0): 
        heading = 0
        
    heading = -heading + offSet
    if heading > 360:
        heading = heading - 360;
    elif heading < 0:
        heading = heading + 360;
        
    return heading

# Navigation
def getSteering(currHead, currLat, currLon, targLat, targLon): # Steering angle towards target locaation wrt target location
    # currHead must be from -360 to 360;
    x = math.cos(np.deg2rad(targLat)) * math.sin(np.deg2rad(currLon-targLon));
    y = math.cos(np.deg2rad(currLat)) * math.sin(np.deg2rad(targLat)) - math.sin(np.deg2rad(currLat))* math.cos(np.deg2rad(targLat)) * math.cos(np.deg2rad(currLon-targLon));
    mag = currHead;
    steering = -np.rad2deg(math.atan2(x,y)) - mag;
    if steering >= 180:
        steering = steering - 360;
    elif steering <= -180:
        steering = steering + 360;
    return steering
        
def getDistance(curLat, curLon, lat2, lon2):  # generally used geo measurement function
    R = 6378.137; # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - curLat * math.pi / 180
    dLon = lon2 * math.pi / 180 - curLon * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(curLat * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c;
    return d * 1000; # meters

def mapf(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def differentialDrive(steeringAngle, driveSpeed):
    # steeringAngle must be from -180 to 180.

    leftDriveSpeed = driveSpeed;
    rightDriveSpeed = driveSpeed;
    if (steeringAngle <= 0):
        L = leftDriveSpeed - mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed
        
    if (steeringAngle < -90) and (steeringAngle >= -180):
        L = leftDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed
      
    
    if (steeringAngle > 0) and steeringAngle <= 90:
        L = leftDriveSpeed
        R = rightDriveSpeed - mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down
    
    if (steeringAngle > 90) and steeringAngle > 90:
        L = leftDriveSpeed
        R = rightDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down
        

    L_R_velocity = [L, R,mapf(abs(steeringAngle), 0,180, 0,driveSpeed)]
    return L_R_velocity

# robot.stop

try:
    while True:
        imu.readSensor() # Read IMU Sensor
        imu.computeOrientation()
        
        received_data = (str)(ser.readline()) #read NMEA string received
        GPGGA_data_available = received_data.find(gpgga_info)   #check for NMEA GPGGA string
        
        if (GPGGA_data_available>0): # when GPS is active
            GPGGA_buffer = received_data.split("$GPGGA,",1)[1]  #store data coming after “$GPGGA,” string
            NMEA_buff = (GPGGA_buffer.split(','))
            nmea_time = []
            nmea_latitude = []
            nmea_longitude = []
            nmea_time = float(NMEA_buff[0]) + 53000                    #extract time from GPGGA string
            nmea_latitude = NMEA_buff[1]                #extract latitude from GPGGA string
            nmea_latitude_dir = NMEA_buff[2]                #Lat direction
            nmea_longitude = NMEA_buff[3]               #extract longitude from GPGGA string
            nmea_long_dir = NMEA_buff[4]                #Long direction  

            GPS_Fix = NMEA_buff[5]
            satellite_count = NMEA_buff[6]
            Alt = NMEA_buff[8]

            lat = float(nmea_latitude)
            lat = convert_to_degrees(lat)
            longi = float(nmea_longitude)
            longi = convert_to_degrees(longi)
        
            currentHeading1 = currentHeading()
            print("currentHeading:",currentHeading1)
            currentLat = lat
            currentLon = longi
            
# #             targetLat = 17.747145
#             targetLat = 17.747200833333334
# #             targetLon = 83.2378320
#             targetLon = 83.23780333333335
    
            targetLat = 17.7472805
            targetLon =83.23778333333333
    
#     17.747200833333334 , 83.23780333333335
    
            print('Time: ',nmea_time)
            print('Current Location: ',currentLat,',',currentLon)
            

            driveSpeed = 0.2

            steerAng = getSteering(currentHeading1, currentLat, currentLon, targetLat, targetLon)
            distFromTraget = getDistance(currentLat, currentLon, targetLat, targetLon)
            
            print('Steering Angle: ',steerAng)
            print('Distance from Target: ',distFromTraget)

            if distFromTraget > 3:
                drive = differentialDrive(steerAng, driveSpeed)
                LWSpeed = drive[0]
                RWSpeed = drive[1]
                print('Speed:\t',LWSpeed,'\t,',RWSpeed)
                robot.set_motors(LWSpeed,RWSpeed)

            else:
                print("Goal Reached")
                robot.set_motors(0,0)
                break     
        
except KeyboardInterrupt:
    robot.stop()
    sys.exit(0)



#  robot.set_motors(0.5,0.5)

currentHeading: 285.69753416874653
Time:  157906.0
Current Location:  17.747289833333333 , 83.23759266666666
Steering Angle:  167.24468015915878
Distance from Target:  20.241510334451934
Speed:	 0.2 	, -0.1716548447981306
currentHeading: 284.755979378028
Time:  157907.0
Current Location:  17.7472935 , 83.23759083333334
Steering Angle:  169.29986973395913
Distance from Target:  20.46044304913485
Speed:	 0.2 	, -0.17622193274213138
currentHeading: 284.755979378028
Time:  157908.0
Current Location:  17.747297166666662 , 83.237592
Steering Angle:  170.46975914118366
Distance from Target:  20.3701763268123
Speed:	 0.2 	, -0.17882168698040812
currentHeading: 293.40080140987925
Time:  157909.0
Current Location:  17.74730116666667 , 83.23759166666665
Steering Angle:  163.05834587028272
Distance from Target:  20.450664311160086
Speed:	 0.2 	, -0.16235187971173937
currentHeading: 301.6661390571901
Time:  157910.0
Current Location:  17.74730483333333 , 83.23759399999999
Steering Angle:  156.01907

In [4]:
robot.stop()

In [18]:
def mapf(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def differentialDrive(steeringAngle, driveSpeed):
    # steeringAngle must be from -180 to 180.

    leftDriveSpeed = driveSpeed;
    rightDriveSpeed = driveSpeed;
    if (steeringAngle <= 0):
        L = leftDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed

    
    if (steeringAngle > 0):
        L = leftDriveSpeed
        R = rightDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down

    L_R_velocity = [L, R,mapf(abs(steeringAngle), 0,180, 0,driveSpeed)]
    return L_R_velocity

differentialDrive(-40,2)


[1.1111111111111112, 2, 0.4444444444444444]